In [1]:
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('image1.jpg')
image2 = cv2.imread('image2.jpg')

height, width, _ = image1.shape
canvas_height = height * 3
canvas_width = width * 3
green_canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
green_canvas[:] = (0, 0, 0)  # Set the canvas to green
y_offset = (canvas_height - height) // 2
x_offset = (canvas_width - width) // 2
green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = image1
image1 = green_canvas


height, width, _ = image2.shape
canvas_height = height * 3
canvas_width = width * 3
green_canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
green_canvas[:] = (0, 0, 0)  # Set the canvas to green
y_offset = (canvas_height - height) // 2
x_offset = (canvas_width - width) // 2
green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = image2
image2 = green_canvas





# Convert images to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Detect ORB keypoints and descriptors
orb = cv2.ORB_create()
keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

# Match descriptors using BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# Sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find the affine transformation matrix
matrix, _ = cv2.estimateAffine2D(points2, points1)

# Apply the affine transformation
result = cv2.warpAffine(image2, matrix, (image1.shape[1], image1.shape[0]))

# Overlay the images
cv2.imwrite('result.jpg', result)
output = cv2.addWeighted(image1, 0.5, result, 0.5, 0)

# Save the result
cv2.imwrite('output.jpg', output)

True